In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster as skc
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_samples
from sklearn.model_selection import train_test_split
from sklearn import tree 
from math import log

In [2]:
path = r'E:\\学业\\IPS\\组会\\task_2\\03_trainingData_B1.csv'

In [3]:
with open(path,encoding='utf-8')as f:
    data = np.loadtxt(path, dtype=str, delimiter=',')

train_df = pd.read_csv(path, header=0)
train_df = train_df[:]
train_df = train_df.replace('*',-110)
train_df['label']=0

C:\Users\Codex\AppData\Local\Temp\ipykernel_18460\2629636238.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['label']=0


In [4]:
train_df

,ID,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,...,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,checkSum,label
0,1,-110,-110,-88,-110,-110,-110,-110,-110,-110,...,35.094452,1,1,218,2,11,13,1369911162,1:-7685.72349999845:4864929.89029999,0
1,2,-110,-110,-87,-110,-110,-110,-110,-110,-110,...,35.094452,1,1,218,2,11,13,1369911145,1:-7685.72349999845:4864929.89029999,0
2,3,-110,-110,-88,-110,-110,-110,-110,-110,-110,...,35.094452,1,1,218,2,11,13,1369911148,1:-7685.72349999845:4864929.89029999,0
3,4,-110,-110,-87,-110,-110,-110,-110,-110,-110,...,35.094452,1,1,218,2,11,13,1369911151,1:-7685.72349999845:4864929.89029999,0
4,5,-110,-110,-87,-110,-110,-110,-110,-110,-110,...,35.094452,1,1,218,2,11,13,1369911155,1:-7685.72349999845:4864929.89029999,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,5244,-110,-110,-110,-88,-110,-110,-110,-86,-86,...,91.295152,4,1,109,2,1,14,1371054764,4:-7587.10229999944:4864986.09099999,0
5244,5245,-110,-110,-110,-88,-110,-110,-110,-86,-86,...,91.295152,4,1,109,2,1,14,1371054765,4:-7587.10229999944:4864986.09099999,0
5245,5246,-110,-110,-110,-90,-110,-110,-110,-86,-86,...,91.295152,4,1,109,2,1,14,1371054765,4:-7587.10229999944:4864986.09099999,0
5246,5247,-110,-110,-110,-90,-110,-110,-110,-86,-86,...,91.295152,4,1,109,2,1,14,1371054765,4:-7587.10229999944:4864986.09099999,0


In [5]:
#手动打标签，如果是坐标轴右上方的为标签1，左下方的为标签0

train_df.label[(train_df['LONGITUDE']<60) & (train_df['LATITUDE']<60)] =1
train_df.label[(train_df['LONGITUDE']>60) & (train_df['LATITUDE']>60)] =0

C:\Users\Codex\AppData\Local\Temp\ipykernel_18460\3633591595.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.label[(train_df['LONGITUDE']<60) & (train_df['LATITUDE']<60)] =1
C:\Users\Codex\AppData\Local\Temp\ipykernel_18460\3633591595.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.label[(train_df['LONGITUDE']>60) & (train_df['LATITUDE']>60)] =0


In [6]:
train_df_features = scale(np.asarray(train_df.iloc[:,1:201]).astype(float), axis =1)

In [9]:
train_df_label = np.asarray(train_df.iloc[:,211:212])

In [14]:
def calcShannonEnt(dataSet):
    numEntries = len(train_df_features)
    labelCounts = {}#计算数据集中数据的个数，这里是5个
    for featVec in train_df_label:#通过循环，featVec=[1,1,'yes']
        currentLabel = str(featVec) #取出这一行的最后一个数，yes or no
        if currentLabel not in labelCounts.keys():
           labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1  #yes为2 no为3
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt#计算香农熵

In [15]:
calcShannonEnt(train_df_features)

0.9999995809393182

In [10]:
numEntries = len(train_df_features)

In [11]:
numEntries

5248